In [3]:
import pandas as pd
import yaml
from math import isnan

In [9]:
VERSION = "2.0"

In [10]:
df = pd.read_csv('./freshaf-{}.csv'.format(VERSION))
df = df.drop(columns=[c for c in df.columns if c.startswith('Unnamed')])
entries = df.to_dict('records')

In [11]:
import base64
import hashlib

def short_hash(text):
    hasher = hashlib.md5(text.encode('utf-8'))
    return str(base64.b64encode(hasher.digest()[:8]).decode('utf-8'))

In [12]:
default_levels = {
    'bronze': 200,
    'silver': 500,
    'gold': 1000
}

thresholds = {
    'velocity': default_levels.copy(),
    'security': default_levels.copy(),
    'quality': default_levels.copy(),
    'resilience': default_levels.copy()
}

In [15]:
questions = []
for entry in entries:
    scores = {}
    question = {
        'id': entry['id'] if 'id' in entry else short_hash(entry['summary']),
        'summary': entry['summary'],
        'points': scores
    }
    
    if isinstance(entry['description'], str):
        question['description'] = entry['description']
    
    has_points = False
    for category in ['velocity', 'quality', 'security', 'resilience']:
        if entry[category] and not isnan(entry[category]):
            scores[category] = int(entry[category])
            has_points = True
            
    if has_points:
        questions.append(question)
    
result = {
    'version': VERSION,
    'thresholds': thresholds,
    'questions': questions
}

with open('./freshaf-{}.yml'.format(VERSION), 'w') as file:
    yaml.dump(result, file, default_flow_style=False, sort_keys=False)